### 1. house 데이터에 적용한 linear regression, support vector machine(선형), 그리고 RANSAC에 의해 추정된 선형모형을 제시하고, RANSAC(그러므로 outlier가 제거된 OLS 선형모형)과의 비교를 통해 linear regression 모형과 support vector machine을 설명하라.

In [32]:
import pandas as pd
from sklearn import datasets
boston_data = datasets.load_boston()

In [33]:
print(boston_data.keys())

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])


In [34]:
print(boston_data.data.shape)

(506, 13)


In [35]:
print(boston_data.feature_names)

['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']


In [36]:
print(boston_data.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

In [37]:
house = pd.DataFrame(boston_data.data)
house.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [38]:
house.columns = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']
house.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [39]:
house.shape

(506, 13)

In [40]:
house['MEDV'] = boston_data.target
house['MEDV'].head()

0    24.0
1    21.6
2    34.7
3    33.4
4    36.2
Name: MEDV, dtype: float64

In [41]:
house.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [42]:
house.shape

(506, 14)

In [43]:
import numpy as np
house['LLSTAT'] = np.log(house['LSTAT'])
house['LINDUS'] = np.log(house['INDUS'])
house.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV,LLSTAT,LINDUS
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,1.605430,0.837248
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,2.212660,1.955860
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,1.393766,1.955860
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,1.078410,0.779325
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,1.673351,0.779325


### Linear Regression

In [44]:
y = house['MEDV'].values
house = house.drop(['LSTAT', 'INDUS', 'MEDV'], axis=1)
X = house.values

In [54]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=3)

from sklearn.linear_model import LinearRegression
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)

print(model_lr.coef_)
print(model_lr.intercept_)

[-1.45361308e-01  1.34472640e-02  3.07365918e+00 -1.05183888e+01
  2.28890141e+00  2.69538428e-02 -1.05812069e+00  2.71778454e-01
 -6.24156416e-03 -7.36156503e-01  1.04555345e-02 -9.30348997e+00
 -1.25084878e+00]
50.97776631422154


In [55]:
y_train_pred = model_lr.predict(X_train)
y_test_pred = model_lr.predict(X_test)

In [56]:
from sklearn.metrics import mean_squared_error
print('MSE train: %.3f' % (mean_squared_error(y_train, y_train_pred)))
print('MSE test: %.3f' % (mean_squared_error(y_test, y_test_pred)))

MSE train: 17.836
MSE test: 17.572


In [57]:
from sklearn.metrics import r2_score
print('R^2 train: %.3f' % (r2_score(y_train, y_train_pred)))
print('R^2 test: %.3f' % (r2_score(y_test, y_test_pred)))

R^2 train: 0.795
R^2 test: 0.777


### RANSAC

In [61]:
from sklearn.linear_model import RANSACRegressor
rans = RANSACRegressor(max_trials=100, min_samples=45, loss='absolute_loss', residual_threshold=5.0, random_state=3)
rans.fit(X, y)

print(rans.estimator_.coef_)
print(rans.estimator_.intercept_)

[-1.03597597e-01 -2.53122572e-03  2.64947511e+00 -5.25252810e+00
  4.63348417e+00 -1.23268389e-02 -9.65923504e-01  3.79132943e-02
 -6.06678895e-03 -5.49043343e-01  7.48396806e-03 -5.74805694e+00
 -1.16020988e+00]
25.910031742421207


In [62]:
y_train_pred = rans.predict(X_train)
y_test_pred = rans.predict(X_test)

In [63]:
print('MSE train: %.3f' % (mean_squared_error(y_train, y_train_pred)))
print('MSE test: %.3f' % (mean_squared_error(y_test, y_test_pred)))

MSE train: 22.191
MSE test: 18.448


In [64]:
print('R^2 train: %.3f' % (r2_score(y_train, y_train_pred)))
print('R^2 test: %.3f' % (r2_score(y_test, y_test_pred)))

R^2 train: 0.745
R^2 test: 0.765


### SVM 선형 회귀

In [65]:
from sklearn.svm import SVR
svl = SVR(kernel='linear', C=1.0, epsilon=0.1)
svl.fit(X_train, y_train)

print(svl.coef_)
print(svl.intercept_)

[[-0.13856545  0.01259793  1.34141245 -1.67426492  4.1974654  -0.00779628
  -0.71869784  0.16539966 -0.00705687 -0.6880149   0.01395138 -5.55243046
  -1.26708833]]
[24.9837281]


In [66]:
y_train_predsvl = svl.predict(X_train)
y_test_predsvl = svl.predict(X_test)

In [67]:
print('MSE train: %.3f' % (mean_squared_error(y_train, y_train_predsvl)))
print('MSE test: %.3f' % (mean_squared_error(y_test, y_test_predsvl)))

MSE train: 21.041
MSE test: 19.506


In [68]:
print('R^2 train: %.3f' % (r2_score(y_train, y_train_predsvl)))
print('R^2 test: %.3f' % (r2_score(y_test, y_test_predsvl)))

R^2 train: 0.758
R^2 test: 0.752


### 지수표현 없애기

In [69]:
import numpy as np
np.set_printoptions(precision=6, suppress=True)

In [70]:
print(model_lr.coef_)
print(model_lr.intercept_)

[ -0.145361   0.013447   3.073659 -10.518389   2.288901   0.026954
  -1.058121   0.271778  -0.006242  -0.736157   0.010456  -9.30349
  -1.250849]
50.97776631422154


In [71]:
print(rans.estimator_.coef_)
print(rans.estimator_.intercept_)

[-0.103598 -0.002531  2.649475 -5.252528  4.633484 -0.012327 -0.965924
  0.037913 -0.006067 -0.549043  0.007484 -5.748057 -1.16021 ]
25.910031742421207


In [72]:
print(svl.coef_)
print(svl.intercept_)

[[-0.138565  0.012598  1.341412 -1.674265  4.197465 -0.007796 -0.718698
   0.1654   -0.007057 -0.688015  0.013951 -5.55243  -1.267088]]
[24.983728]
